In [6]:
import pandas as pd
from selenium import webdriver
import glob
import os
from tqdm import tqdm
import re
df_annotated_clean = pd.read_excel("df_annotated_cleaned.xlsx")
geckodriver_path=r'C:\Users\Walid\Desktop\Selenium\geckodriver.exe'

# I. First BaseLine based on the Metadata

Each HTML page has a head that contains information about the page and instructions for web browsers and search engine web crawlers.

Check this page to know more about the Metadata https://html.com/document/metadata/


From the Metadata we get the title and the description

## 1. Single Example

In [42]:
driver = webdriver.Firefox(executable_path=geckodriver_path)

In [59]:
i = 35
url_local = df_annotated['URL_Local'][i]
driver.get(os.getcwd()+'\\'+url_local)
title = driver.title
description = driver.find_element_by_xpath("//*[@name='description']").get_attribute("content")
print(title)
print("--------------------------------------------------------------")
print(description)

Meuble bas "Garcinia" blanc brillant l.40 x h.72 x p.57 cm 1 porte + 1 tiroir - Brico Dépôt
--------------------------------------------------------------
 - Brico Dépôt


## 2. What's in the metadata

In [26]:
metadata = driver.find_elements_by_xpath("//meta")
metadata = [driver.execute_script('var items = {}; for (index = 0; index < arguments[0].attributes.length; ++index) { items[arguments[0].attributes[index].name] = arguments[0].attributes[index].value }; return items;', metadata[i]) for i in range(len(metadata))]
# metadata

In [77]:
def firs_baseline(df_annotated_clean, geckodriver_path=r'C:\Users\Walid\Desktop\Selenium\geckodriver.exe'):
    description_missing = 0
    price_missing = 0
    out = []
    driver = webdriver.Firefox(executable_path=geckodriver_path)
    
    for i in tqdm(range(df_annotated_clean.shape[0])):
        url_local = df_annotated_clean['URL_Local'][i]
        driver.get(os.getcwd()+'\\'+url_local)

        title = driver.title
        try:
            description = driver.find_element_by_xpath("//*[@name='description']").get_attribute("content")
        except:
            description_missing += 1
            description = ""
        element_price = driver.find_elements_by_xpath("//*[contains(@property,'price')]")
        for element in element_price:
            try :
                price = float(element.replace(',','.'))
                break
            except:
                price =""
                price_missing+=1
                
        
        out.append([url_local, df_annotated_clean['URL_Live'][i], df_annotated_clean['URL_Parent'][i], title, description, price])
        
    df_out = pd.DataFrame(out, columns=['URL_Local','URL_Live','URL_Parent','Title_Content','Description_Content','Price_content'])
    df_out.to_excel('First_Baseline.xlsx', index=False)
    driver.close()
    return df_out, df_annotated_clean.shape[0], '{:.1%}'.format(1), '{:.1%}'.format(1-description_missing/df_annotated_clean.shape[0]), '{:.1%}'.format(1-price_missing/df_annotated_clean.shape[0])

In [78]:
df_out, nbr_observations, title_availability, description_availability, price_availability = firs_baseline(df_anhnoated_clean)

100%|██████████████████████████████████████████████████████████████████████████████████| 57/57 [00:22<00:00,  2.57it/s]


## 3. What's wrong with the first baseline

    1. The metadata content is not the same content displayed on the webpage for instance we can find that the title are not exactly the same, however they contains the same information. Same for description

In [76]:
print("The number of observation is":, nbr_observations)
print("Title is available: ", title_availability )
print("Description is available: ", description_availability)   
print("Price is available: ", price_availability)

Tile is available:  100.0%
Description is available:  98.2%
Price is available:  40.4%


# II. Second BaseLine based on the vocabulary related to each element

We look for element that are supposed to contain title, description and price based on the tag names, class names....

In [79]:
from requests_testadapter import Resp
import extruct
from w3lib.html import get_base_url
import requests

class LocalFileAdapter(requests.adapters.HTTPAdapter):
    def build_response_from_file(self, request):
        file_path = request.url[7:]
        with open(file_path, 'rb') as file:
            buff = bytearray(os.path.getsize(file_path))
            file.readinto(buff)
            resp = Resp(buff)
            r = self.build_response(request, resp)

            return r

    def send(self, request, stream=False, timeout=None,
             verify=True, cert=None, proxies=None):

        return self.build_response_from_file(request)

def get_html(url):
    """Get raw HTML from a URL."""
    requests_session = requests.session()
    requests_session.mount('file://', LocalFileAdapter())
    r = requests_session.get('file://'+urls[1])
    return r.text

In [9]:
# driver = webdriver.Firefox(executable_path=geckodriver_path)

def second_baseline(df_annotated_clean, geckodriver_path=r'C:\Users\Walid\Desktop\Selenium\geckodriver.exe'):
    price_missing = 0
    title_missing = 0
    out = []
    driver = webdriver.Firefox(executable_path=geckodriver_path)
    
    for i in tqdm(range(df_annotated_clean.shape[0])):
        url_local = df_annotated_clean['URL_Local'][i]
        driver.get(os.getcwd()+'\\'+url_local)
        
        title_elements = driver.find_elements_by_tag_name("h1")
        title_txt = ""
        for element in title_elements:
            if len(element.text) > 0 :
                title_txt = element.text
            break
        
        price_elements = driver.find_elements_by_xpath("//*[contains(@class,'price')]")
        price = ""
        for element in price_elements:
            price_txt = element.text
            numbers = re.findall(r"[-+]?\d*\.\d+|\d+ ", price_txt.replace(',','.'))
            if numbers!=[]:
                price = numbers[0]
                break
        out.append([url_local, df_annotated_clean['URL_Live'][i], df_annotated_clean['URL_Parent'][i], title_txt, price])
        
    df_out = pd.DataFrame(out, columns=['URL_Local','URL_Live','URL_Parent','Title_Content','Price_content'])
    df_out.to_excel('Second_Baseline.xlsx', index=False)
    driver.close()
    return df_out

In [10]:
second_baseline = second_baseline(df_annotated_clean)

100%|██████████████████████████████████████████████████████████████████████████████████| 57/57 [00:23<00:00,  2.47it/s]
